# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## All CpG

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Gastro
df = pd.read_csv('Gastro_with_MethylomeDetails.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Gastro
(458, 34012)


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,SCOPA_Gastro_Category,cg03441945,cg21094410,...,cg12522722,cg11255513,cg09615521,cg12658517,cg02150941,cg22992837,cg02699881,cg05225390,cg01481208,cg08634588
0,200973410159_R03C01,3000,2,2,0,69.1,4,0,0.892410,0.015756,...,0.909955,0.043985,0.054816,0.904326,0.135138,0.742557,0.029678,0.079017,0.894593,0.902949
1,200991620021_R01C01,3001,1,1,2,65.1,4,0,0.908813,0.011350,...,0.900398,0.043525,0.043829,0.899376,0.078902,0.754703,0.030224,0.075161,0.909315,0.903716
2,200991620021_R03C01,3002,2,1,2,67.6,4,1,0.922532,0.013309,...,0.931750,0.051855,0.039594,0.921650,0.074618,0.843395,0.020945,0.058536,0.872011,0.890447


In [3]:
df1 = df.drop(['Sentrix','PATNO','HYS','Gender','SCOPA_Gastro_Category','Age','AgeCate'], axis=1)
df1.head(2)

,APPRDX,cg03441945,cg21094410,cg22926093,cg25025609,cg16704802,cg06228737,cg00220102,cg19016494,cg12247200,...,cg12522722,cg11255513,cg09615521,cg12658517,cg02150941,cg22992837,cg02699881,cg05225390,cg01481208,cg08634588
0,2,0.892410,0.015756,0.011101,0.018491,0.020476,0.962482,0.677222,0.029560,0.929821,...,0.909955,0.043985,0.054816,0.904326,0.135138,0.742557,0.029678,0.079017,0.894593,0.902949
1,1,0.908813,0.011350,0.012197,0.022372,0.018128,0.945195,0.730494,0.017911,0.921435,...,0.900398,0.043525,0.043829,0.899376,0.078902,0.754703,0.030224,0.075161,0.909315,0.903716


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.485
2
Mean accuracy: 0.49000000000000005
3
Mean accuracy: 0.53
4
Mean accuracy: 0.51
5
Mean accuracy: 0.62
6
Mean accuracy: 0.5899999999999999
7
Mean accuracy: 0.5149999999999999
8
Mean accuracy: 0.5549999999999999
9
Mean accuracy: 0.53
10
Mean accuracy: 0.5
11
Mean accuracy: 0.48999999999999994
12
Mean accuracy: 0.5
13
Mean accuracy: 0.48
14
Mean accuracy: 0.49499999999999994
15
Mean accuracy: 0.4950000000000001
16
Mean accuracy: 0.555
17
Mean accuracy: 0.5799999999999998
18
Mean accuracy: 0.5099999999999999
19
Mean accuracy: 0.42000000000000004
20
Mean accuracy: 0.505
21
Mean accuracy: 0.5249999999999999
22
Mean accuracy: 0.505
23
Mean accuracy: 0.54
24
Mean accuracy: 0.535
25
Mean accuracy: 0.5349999999999999
26
Mean accuracy: 0.51
27
Mean accuracy: 0.48
28
Mean accuracy: 0.5449999999999999
29
Mean accuracy: 0.525
30
Mean accuracy: 0.48999999999999994
31
Mean accuracy: 0.51
32
Mean accuracy: 0.43
33
Mean accuracy: 0.5250000000000001
34
Mean accuracy: 0.46000000000

In [5]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Gastro_AllCpG_100RF.csv", index = False)

0.5142500000000001


## RanFor 1000 times. Samples = 50

In [6]:
df2 = df1.drop (['APPRDX'], axis=1)

In [7]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.41
2
Mean accuracy: 0.59
3
Mean accuracy: 0.5149999999999999
4
Mean accuracy: 0.5250000000000001
5
Mean accuracy: 0.49499999999999994
6
Mean accuracy: 0.5050000000000001
7
Mean accuracy: 0.5
8
Mean accuracy: 0.51
9
Mean accuracy: 0.51
10
Mean accuracy: 0.545
11
Mean accuracy: 0.54
12
Mean accuracy: 0.49000000000000005
13
Mean accuracy: 0.56
14
Mean accuracy: 0.525
15
Mean accuracy: 0.505
16
Mean accuracy: 0.47000000000000003
17
Mean accuracy: 0.5050000000000001
18
Mean accuracy: 0.515
19
Mean accuracy: 0.52
20
Mean accuracy: 0.5999999999999999
21
Mean accuracy: 0.53
22
Mean accuracy: 0.4800000000000001
23
Mean accuracy: 0.525
24
Mean accuracy: 0.51
25
Mean accuracy: 0.525
26
Mean accuracy: 0.40499999999999997
27
Mean accuracy: 0.54
28
Mean accuracy: 0.48999999999999994
29
Mean accuracy: 0.495
30
Mean accuracy: 0.53
31
Mean accuracy: 0.525
32
Mean accuracy: 0.5399999999999999
33
Mean accuracy: 0.6000000000000001
34
Mean accuracy: 0.5750000000000001
35
Mean accuracy: 0

In [8]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Gastro_Rand50CpG_100RF.csv", index = False)

0.5080500000000001


## RanFor 100 times. Samples = 98

In [9]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(98, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.44000000000000006
2
Mean accuracy: 0.565
3
Mean accuracy: 0.45
4
Mean accuracy: 0.58
5
Mean accuracy: 0.51
6
Mean accuracy: 0.4999999999999999
7
Mean accuracy: 0.49000000000000005
8
Mean accuracy: 0.56
9
Mean accuracy: 0.51
10
Mean accuracy: 0.57
11
Mean accuracy: 0.53
12
Mean accuracy: 0.51
13
Mean accuracy: 0.5599999999999999
14
Mean accuracy: 0.53
15
Mean accuracy: 0.515
16
Mean accuracy: 0.45
17
Mean accuracy: 0.49499999999999994
18
Mean accuracy: 0.595
19
Mean accuracy: 0.545
20
Mean accuracy: 0.545
21
Mean accuracy: 0.575
22
Mean accuracy: 0.48999999999999994
23
Mean accuracy: 0.53
24
Mean accuracy: 0.5
25
Mean accuracy: 0.5199999999999999
26
Mean accuracy: 0.45499999999999996
27
Mean accuracy: 0.5499999999999999
28
Mean accuracy: 0.4700000000000001
29
Mean accuracy: 0.48
30
Mean accuracy: 0.53
31
Mean accuracy: 0.5800000000000001
32
Mean accuracy: 0.55
33
Mean accuracy: 0.5599999999999999
34
Mean accuracy: 0.505
35
Mean accuracy: 0.45
36
Mean accuracy: 0.51499

In [10]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Gastro_Rand98CpG_100RF.csv", index = False)

0.5126999999999998


## RanFor Top 50 Boruta 100 times. Samples = 50

In [11]:
df = pd.read_csv('Gastro_Boruta_Top50_Data.csv')
print(df.shape)
df.head(2)

(458, 52)


,PATNO,APPRDX,cg12452675,cg25774148,cg04926619,cg02744123,cg11351331,cg07250622,cg21043558,cg11697588,...,cg05142211,cg22152677,cg06734098,cg12220605,cg00875272,cg13468400,cg08096908,cg17223615,cg16199144,cg19371224
0,3000,2,0.047168,0.940005,0.627342,0.014085,0.895471,0.058003,0.638077,0.051262,...,0.019571,0.932195,0.340984,0.117280,0.032753,0.895386,0.937764,0.822963,0.033485,0.895183
1,3001,1,0.040326,0.915349,0.632705,0.013944,0.898821,0.032852,0.667449,0.076483,...,0.035018,0.927876,0.050031,0.082447,0.026473,0.812987,0.905542,0.814484,0.027345,0.751572


In [12]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg12452675,cg25774148,cg04926619,cg02744123,cg11351331,cg07250622,cg21043558,cg11697588,cg12510028,...,cg05142211,cg22152677,cg06734098,cg12220605,cg00875272,cg13468400,cg08096908,cg17223615,cg16199144,cg19371224
0,2,0.047168,0.940005,0.627342,0.014085,0.895471,0.058003,0.638077,0.051262,0.037114,...,0.019571,0.932195,0.340984,0.117280,0.032753,0.895386,0.937764,0.822963,0.033485,0.895183
1,1,0.040326,0.915349,0.632705,0.013944,0.898821,0.032852,0.667449,0.076483,0.029810,...,0.035018,0.927876,0.050031,0.082447,0.026473,0.812987,0.905542,0.814484,0.027345,0.751572


In [13]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.645
2
Mean accuracy: 0.635
3
Mean accuracy: 0.6649999999999999
4
Mean accuracy: 0.67
5
Mean accuracy: 0.5950000000000001
6
Mean accuracy: 0.67
7
Mean accuracy: 0.62
8
Mean accuracy: 0.6849999999999999
9
Mean accuracy: 0.615
10
Mean accuracy: 0.635
11
Mean accuracy: 0.6849999999999999
12
Mean accuracy: 0.635
13
Mean accuracy: 0.635
14
Mean accuracy: 0.6100000000000001
15
Mean accuracy: 0.67
16
Mean accuracy: 0.6500000000000001
17
Mean accuracy: 0.675
18
Mean accuracy: 0.6199999999999999
19
Mean accuracy: 0.595
20
Mean accuracy: 0.6300000000000001
21
Mean accuracy: 0.74
22
Mean accuracy: 0.6249999999999999
23
Mean accuracy: 0.64
24
Mean accuracy: 0.63
25
Mean accuracy: 0.6599999999999999
26
Mean accuracy: 0.655
27
Mean accuracy: 0.67
28
Mean accuracy: 0.7
29
Mean accuracy: 0.645
30
Mean accuracy: 0.655
31
Mean accuracy: 0.655
32
Mean accuracy: 0.6
33
Mean accuracy: 0.655
34
Mean accuracy: 0.6399999999999999
35
Mean accuracy: 0.6849999999999999
36
Mean accuracy: 0.645
3

In [14]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Gastro_Boruta50CpG_100RF.csv", index = False)

0.6466000000000001


## RanFor Top 50 NMI 100 times. Samples = 50

In [15]:
df = pd.read_csv('Top50NMI_Gastro_Data.csv')
print(df.shape)
df.head(2)

(458, 52)


,PATNO,APPRDX,cg10783204,cg14547519,cg09879107,cg23584714,cg18764107,cg11670533,cg09347135,cg02449892,...,cg12036621,cg01575096,cg08522144,cg14719352,cg12520249,cg13196143,cg23777848,cg12979803,cg16964348,cg20516176
0,3000,2,0.709340,0.918605,0.621822,0.837783,0.042191,0.849763,0.502712,0.879576,...,0.042321,0.045783,0.930074,0.789685,0.916357,0.596177,0.015398,0.825014,0.021578,0.945058
1,3001,1,0.585492,0.900845,0.727246,0.776498,0.024068,0.790524,0.403945,0.807121,...,0.030171,0.056810,0.893691,0.744240,0.914708,0.521505,0.020775,0.693402,0.021671,0.910851


In [16]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg10783204,cg14547519,cg09879107,cg23584714,cg18764107,cg11670533,cg09347135,cg02449892,cg21040417,...,cg12036621,cg01575096,cg08522144,cg14719352,cg12520249,cg13196143,cg23777848,cg12979803,cg16964348,cg20516176
0,2,0.709340,0.918605,0.621822,0.837783,0.042191,0.849763,0.502712,0.879576,0.896629,...,0.042321,0.045783,0.930074,0.789685,0.916357,0.596177,0.015398,0.825014,0.021578,0.945058
1,1,0.585492,0.900845,0.727246,0.776498,0.024068,0.790524,0.403945,0.807121,0.902297,...,0.030171,0.056810,0.893691,0.744240,0.914708,0.521505,0.020775,0.693402,0.021671,0.910851


In [17]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.635
2
Mean accuracy: 0.7000000000000001
3
Mean accuracy: 0.685
4
Mean accuracy: 0.68
5
Mean accuracy: 0.675
6
Mean accuracy: 0.66
7
Mean accuracy: 0.6650000000000001
8
Mean accuracy: 0.665
9
Mean accuracy: 0.6300000000000001
10
Mean accuracy: 0.635
11
Mean accuracy: 0.635
12
Mean accuracy: 0.715
13
Mean accuracy: 0.6950000000000001
14
Mean accuracy: 0.5750000000000001
15
Mean accuracy: 0.665
16
Mean accuracy: 0.6649999999999999
17
Mean accuracy: 0.72
18
Mean accuracy: 0.715
19
Mean accuracy: 0.6399999999999999
20
Mean accuracy: 0.62
21
Mean accuracy: 0.6449999999999999
22
Mean accuracy: 0.6449999999999999
23
Mean accuracy: 0.6749999999999999
24
Mean accuracy: 0.6699999999999999
25
Mean accuracy: 0.6749999999999999
26
Mean accuracy: 0.595
27
Mean accuracy: 0.66
28
Mean accuracy: 0.6900000000000001
29
Mean accuracy: 0.675
30
Mean accuracy: 0.6799999999999999
31
Mean accuracy: 0.6
32
Mean accuracy: 0.635
33
Mean accuracy: 0.6249999999999999
34
Mean accuracy: 0.710000000

In [18]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Gastro_NMI50CpG_100RF.csv", index = False)

0.6661999999999998


## RanFor Top 50 Boruta + NMI 100 times. Samples = 98

In [19]:
df = pd.read_csv('Gastro_Top50sMerge_Data.csv')
print(df.shape)
df.head(2)

(458, 100)


,PATNO,APPRDX,cg10783204,cg14547519,cg09879107,cg23584714,cg18764107,cg11670533,cg09347135,cg02449892,...,cg26668679,cg22152677,cg06734098,cg12220605,cg00875272,cg13468400,cg08096908,cg17223615,cg16199144,cg19371224
0,3000,2,0.709340,0.918605,0.621822,0.837783,0.042191,0.849763,0.502712,0.879576,...,0.194786,0.932195,0.340984,0.117280,0.032753,0.895386,0.937764,0.822963,0.033485,0.895183
1,3001,1,0.585492,0.900845,0.727246,0.776498,0.024068,0.790524,0.403945,0.807121,...,0.247316,0.927876,0.050031,0.082447,0.026473,0.812987,0.905542,0.814484,0.027345,0.751572


In [20]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg10783204,cg14547519,cg09879107,cg23584714,cg18764107,cg11670533,cg09347135,cg02449892,cg21040417,...,cg26668679,cg22152677,cg06734098,cg12220605,cg00875272,cg13468400,cg08096908,cg17223615,cg16199144,cg19371224
0,2,0.709340,0.918605,0.621822,0.837783,0.042191,0.849763,0.502712,0.879576,0.896629,...,0.194786,0.932195,0.340984,0.117280,0.032753,0.895386,0.937764,0.822963,0.033485,0.895183
1,1,0.585492,0.900845,0.727246,0.776498,0.024068,0.790524,0.403945,0.807121,0.902297,...,0.247316,0.927876,0.050031,0.082447,0.026473,0.812987,0.905542,0.814484,0.027345,0.751572


In [21]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.665
2
Mean accuracy: 0.71
3
Mean accuracy: 0.6900000000000001
4
Mean accuracy: 0.6950000000000001
5
Mean accuracy: 0.6300000000000001
6
Mean accuracy: 0.6950000000000001
7
Mean accuracy: 0.6849999999999999
8
Mean accuracy: 0.65
9
Mean accuracy: 0.635
10
Mean accuracy: 0.635
11
Mean accuracy: 0.66
12
Mean accuracy: 0.7350000000000001
13
Mean accuracy: 0.605
14
Mean accuracy: 0.67
15
Mean accuracy: 0.6599999999999999
16
Mean accuracy: 0.67
17
Mean accuracy: 0.725
18
Mean accuracy: 0.6549999999999999
19
Mean accuracy: 0.61
20
Mean accuracy: 0.6699999999999999
21
Mean accuracy: 0.7250000000000001
22
Mean accuracy: 0.66
23
Mean accuracy: 0.655
24
Mean accuracy: 0.6849999999999999
25
Mean accuracy: 0.665
26
Mean accuracy: 0.6699999999999999
27
Mean accuracy: 0.67
28
Mean accuracy: 0.7200000000000001
29
Mean accuracy: 0.6950000000000001
30
Mean accuracy: 0.73
31
Mean accuracy: 0.6499999999999999
32
Mean accuracy: 0.64
33
Mean accuracy: 0.6699999999999999
34
Mean accuracy: 0

In [22]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Gastro_Top50sCpG98_100RF.csv", index = False)

0.6705
